In [1]:
!apt-get install poppler-utils tesseract-ocr -y
!pip install pdfplumber pdf2image pytesseract pillow regex


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 38 not upgraded.
Need to get 186 kB of archives.
After this operation, 697 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.11 [186 kB]
Fetched 186 kB in 1s (218 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 126675 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.11_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.11) ...
Setting up poppler-utils (22.02.0-2ubuntu0.11) ...
Processing triggers for man-db (2.10.2-1) ...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [6]:
# --- Upload PDF ---
from google.colab import files
uploaded = files.upload()
pdf_path = list(uploaded.keys())[0]
print("Uploaded:", pdf_path)

# --- Imports ---
import re, json, pandas as pd
import pdfplumber, pytesseract
from pdf2image import convert_from_path
from datetime import datetime

# --- Helper functions ---
def pdf_to_text(path):
    """Extract text from PDF with pdfplumber, fallback to OCR if scanned."""
    text = ""
    with pdfplumber.open(path) as pdf:
        for page in pdf.pages:
            page_text = page.extract_text() or ""
            text += page_text + "\n"
    if text.strip():
        return text
    # OCR fallback for scanned PDFs
    images = convert_from_path(path, dpi=300)
    ocr_text = [pytesseract.image_to_string(img) for img in images]
    return "\n".join(ocr_text)


def try_parse_date(s):
    """Attempt to normalize various date formats to YYYY-MM-DD when possible."""
    if not s or not isinstance(s, str):
        return s
    # clean common trailing words
    s = s.strip()
    patterns = ["%d/%m/%Y","%d-%m-%Y","%d %b %Y","%b %d %Y","%Y-%m-%d","%d %b, %Y","%d %B %Y","%d-%b-%Y"]
    for fmt in patterns:
        try:
            return datetime.strptime(s, fmt).strftime("%Y-%m-%d")
        except:
            pass
    # some dates like "17 Nov, 2023 to 16 Dec, 2023" — return as-is for caller to handle
    return s


#Bank Parser

# --- AXIS BANK PARSER ---
def parse_axis(text, clean_lines):
    fields = {}
    m = re.search(r"(?:\{4,}|xxxx|ending\s*in)\s(\d{4})", text, re.I)
    if not m:
        m = re.search(r"\d{4}\{4,}\d{4}|\d{4}\s\d{4}\s*\d{4}\s*(\d{4})", text)
    if m:
        fields["card_last4"] = m.group(1)

    summary_text = ""
    for i, line in enumerate(clean_lines):
        if re.search(r"Payment\s*Summary", line, re.I):
            summary_text = " ".join(clean_lines[i:i+5])
            break

    billing_cycle = re.search(r"(\d{2}/\d{2}/\d{4})\s*-\s*(\d{2}/\d{2}/\d{4})", summary_text)
    if billing_cycle:
        fields["billing_cycle"] = f"{billing_cycle.group(1)} to {billing_cycle.group(2)}"

    due_date = re.search(r"\b(\d{2}/\d{2}/\d{4})\b", summary_text)
    if due_date:
        fields["payment_due_date"] = try_parse_date(due_date.group(1))

    amounts = re.findall(r"[\d,]+\.\d{2}", summary_text)
    if len(amounts) >= 2:
        fields["total_due"] = amounts[-2]
        fields["min_due"] = amounts[-1]
    elif len(amounts) == 1:
        fields["total_due"] = amounts[0]
    else:
        fields["total_due"] = "Not Found"
        fields["min_due"] = "Not Found"

    credit_limit = "Not Found"
    available_credit = "Not Found"
    for i, line in enumerate(clean_lines):
        if re.search(r"Credit\s*Limit", line, re.I):
            nearby_text = " ".join(clean_lines[i:i+4])
            numbers = re.findall(r"\d[\d,]*\.?\d{0,2}", nearby_text)
            nums_clean = [n for n in numbers if re.search(r'\d', n)]
            if len(nums_clean) >= 2:
                credit_limit = nums_clean[0]
                available_credit = nums_clean[1]
            elif len(nums_clean) == 1:
                credit_limit = nums_clean[0]
            break
    fields["credit_limit"] = credit_limit
    fields["available_credit"] = available_credit
    return fields


# --- HDFC BANK PARSER ---
def parse_hdfc(text, clean_lines):
    fields = {}
    m = re.search(r"(?:\{4,}|xxxx|ending\s*in)\s(\d{4})", text, re.I)
    if m:
        fields["card_last4"] = m.group(1)
    m = re.search(r"(?:payment\s*due\s*date|due\s*date)[:\s]*(\d{1,2}[\/\-]\d{1,2}[\/\-]\d{2,4})", text, re.I)
    if m:
        fields["payment_due_date"] = try_parse_date(m.group(1))
    for i, line in enumerate(clean_lines):
        if re.search(r"Total\s*Amount\s*Due", line, re.I):
            nearby_text = " ".join(clean_lines[i:i+3])
            nums = re.findall(r"\d[\d,]*\.\d{2}", nearby_text)
            if nums:
                fields["total_due"] = f"{max(float(n.replace(',', '')) for n in nums):,.2f}"
                break
    m = re.search(r"(?:billing\s*(?:cycle|period)|statement\s*period)[:\s]*(.+)", text, re.I)
    if m:
        fields["billing_cycle"] = m.group(1).strip()
    credit_limit = "Not Found"
    available_credit = "Not Found"
    for i, line in enumerate(clean_lines):
        if re.search(r"Credit\s*Limit", line, re.I):
            nearby_text = " ".join(clean_lines[i:i+4])
            numbers = re.findall(r"\d[\d,]*\.?\d{0,2}", nearby_text)
            nums_clean = [n for n in numbers if re.search(r'\d', n)]
            if len(nums_clean) >= 2:
                available_credit = nums_clean[0]
                credit_limit = nums_clean[1]
            elif len(nums_clean) == 1:
                credit_limit = nums_clean[0]
            break
    fields["credit_limit"] = credit_limit
    fields["available_credit"] = available_credit
    return fields


# --- ICICI / IDFC BANK PARSER ---
def parse_icici_idfc(text, clean_lines):
    fields = {}
    m = re.search(r"(?:\{4,}|xxxx|ending\s*in)\s(\d{4})", text, re.I)
    if m:
        fields["card_last4"] = m.group(1)
    m = re.search(r"(?:payment\s*due\s*date|due\s*date)[:\s]*(\d{1,2}[\/\-]\d{1,2}[\/\-]\d{2,4})", text, re.I)
    if m:
        fields["payment_due_date"] = try_parse_date(m.group(1))
    for i, line in enumerate(clean_lines):
        if re.search(r"Total\s*Amount\s*Due", line, re.I):
            nearby_text = " ".join(clean_lines[i:i+3])
            nums = re.findall(r"\d[\d,]*\.\d{2}", nearby_text)
            if nums:
                fields["total_due"] = f"{max(float(n.replace(',', '')) for n in nums):,.2f}"
                break
    m = re.search(r"(?:billing\s*(?:cycle|period)|statement\s*period)[:\s]*(.+)", text, re.I)
    if m:
        fields["billing_cycle"] = m.group(1).strip()

    def _normalize_amount(token):
        t = token.replace('₹', '').replace('Rs.', '').replace('INR', '')
        t = re.sub(r'[^\d\.,\-]', '', t).strip().replace(',', '')
        try:
            return float(t)
        except:
            return None

    credit_limit = "Not Found"
    available_credit = "Not Found"
    for i, line in enumerate(clean_lines):
        if re.search(r"Credit\s*Limit", line, re.I):
            block = " ".join(clean_lines[i:i+4])
            tokens = re.findall(r"(?:₹|Rs\.)?\s*[0-9][0-9\.,]*\d(?:\.\d{1,2})?", block, re.I)
            amounts = [_normalize_amount(tok) for tok in tokens if _normalize_amount(tok) is not None]
            if len(amounts) >= 2:
                credit_limit = f"{amounts[0]:,.2f}"
                available_credit = f"{amounts[1]:,.2f}"
            elif len(amounts) == 1:
                credit_limit = f"{amounts[0]:,.2f}"
            break
    fields["credit_limit"] = credit_limit
    fields["available_credit"] = available_credit
    return fields


# --- OTHER BANKS PARSER (replaces parse_default) ---
def parse_other_banks(text, clean_lines):
    """
    Generic parser for any bank/issuer other than AXIS/HDFC/ICICI/IDFC.
    - C3 detection: flexible extraction (fintechs, NBFCs included)
    - D1: when co-branded, store only the primary bank name (if found)
    - Fallback: "Unknown Bank"
    """
    fields = {}

    # 1) Known bank patterns map -> canonical bank name (primary bank)
    # Add more patterns if you want better coverage
    known_bank_patterns = {
        r"\b(bank of baroda|bob financial|bobcards|bob financial solutions)\b": "Bank of Baroda",
        r"\b(state bank of india|sbi card|sbi)\b": "State Bank of India",
        r"\b(kotak mahindra|kotak)\b": "Kotak Mahindra Bank",
        r"\b(yes bank)\b": "Yes Bank",
        r"\b(rbl|rbl bank)\b": "RBL Bank",
        r"\b(indusind|indusind bank)\b": "IndusInd Bank",
        r"\b(au small|au bank)\b": "AU Small Finance Bank",
        r"\b(federal bank)\b": "Federal Bank",
        r"\b(citi|citibank)\b": "Citi Bank",
        r"\b(hsbc)\b": "HSBC",
        r"\b(amex|american express|american express)\b": "American Express",
        r"\b(onecard)\b": "OneCard",
        r"\b(slice)\b": "Slice",
        r"\b(amazon pay|amazon)\b": "Amazon",
        r"\b(flipkart)\b": "Flipkart",
        r"\b(boi|bank of india)\b": "Bank of India",
        r"\b(union bank)\b": "Union Bank",
        r"\b(central bank of india)\b": "Central Bank of India",
        r"\b(yes bank)\b": "Yes Bank",
        r"\b(boi financial)\b": "Bank of India"
    }

    text_lower = text.lower()

    detected_bank = None
    for patt, canon in known_bank_patterns.items():
        if re.search(patt, text_lower, re.I):
            # D1: choose primary bank name (canonical)
            detected_bank = canon
            break

    # 2) If not matched by known patterns, attempt C3: pick likely issuer line from top-lines
    if not detected_bank:
        top_chunk = " ".join(clean_lines[:10]).strip()
        # prefer lines containing 'card' or 'credit' with 1-4 words that look like a title
        title_candidates = []
        for i, line in enumerate(clean_lines[:10]):
            # short title-like lines
            if 2 <= len(line.split()) <= 6 and len(line) < 80:
                title_candidates.append(line.strip())
            # prefer lines containing 'card' or 'credit' or 'financial' or 'bank'
            if re.search(r"\b(bank|card|credit|financial|cards|cards)\b", line, re.I):
                title_candidates.insert(0, line.strip())
        # from candidates, pick first one that contains word 'bank' (D1 requires bank)
        chosen = None
        for cand in title_candidates:
            if re.search(r"\bbank\b", cand, re.I):
                chosen = cand
                break
        if chosen:
            # extract the phrase containing 'bank' — normalize spaces
            m = re.search(r"([A-Za-z0-9 &\-']*bank[A-Za-z0-9 &\-']*)", chosen, re.I)
            if m:
                detected_bank = m.group(1).strip()
        else:
            # final attempt: if any known bank word exists anywhere in top_chunk
            m = re.search(r"\b(bank of [A-Za-z]+|[A-Za-z]+ bank|sbi|hdfc|icici|axis|kotak|rbl|indusind|bob)\b", top_chunk, re.I)
            if m:
                detected_bank = m.group(0).strip()

    # 3) Fallback
    fields["bank_detected_name"] = detected_bank if detected_bank else "Unknown Bank"

    # 4) Card last 4
    m = re.search(r"(?:Card\s*No[:\s]*|Card\s*Number[:\s]*|Card\s*:|Card\s+:\s*|card no|card no\.|ending\s*in|xxxx)\D*?(\d{4})", text, re.I)
    if not m:
        # alternative common pattern in many statements: 6XXXXXXX1234 or 468943XXXXXX6743
        m = re.search(r"(\d{4})\s*X{2,}|\d{6}\s*X{2,}(\d{4})", text)
    if m:
        # pick last capturing group that is digits
        g = [g for g in m.groups() if g]
        fields["card_last4"] = g[-1] if g else m.group(1)
    else:
        fields["card_last4"] = "Not Found"

    # 5) Billing cycle (flexible)
    m = re.search(r"(?:Statement\s*Period|Billing\s*(?:Cycle|Period)|Statement\s*Date\s*[:\-])\s*[:\-]?\s*(.+?to.+?\d{4})", text, re.I)
    if m:
        fields["billing_cycle"] = m.group(1).strip()
    else:
        m2 = re.search(r"(\d{1,2}[\/\-]\d{1,2}[\/\-]\d{2,4})\s*(?:to|-)\s*(\d{1,2}[\/\-]\d{1,2}[\/\-]\d{2,4})", text)
        if m2:
            fields["billing_cycle"] = f"{m2.group(1)} to {m2.group(2)}"
        else:
            fields["billing_cycle"] = "Not Found"

    # 6) Payment due date
    m = re.search(r"(?:Payment\s*Due\s*Date|Due\s*Date)[:\s\-]*([0-3]?\d[\/\-][0-3]?\d[\/\-]\d{2,4})", text, re.I)
    if m:
        fields["payment_due_date"] = try_parse_date(m.group(1))
    else:
        # sometimes shown near 'Total Amount Due' block
        near = " ".join(clean_lines[:20])
        m2 = re.search(r"Due\s*Date[:\s]*([0-3]?\d[\/\-][0-3]?\d[\/\-]\d{2,4})", near, re.I)
        fields["payment_due_date"] = try_parse_date(m2.group(1)) if m2 else "Not Found"

    # 7) Total due & Min due
    tot = re.search(r"Total\s*(?:Amount\s*)?Due[:\s₹]*([\d,]+\.\d{2})", text, re.I)
    mind = re.search(r"(?:Minimum|Min)\s*(?:Amount\s*)?Due[:\s₹]*([\d,]+\.\d{2})", text, re.I)
    # fallback: grab prominent currency amounts near top section
    if not tot:
        top_n = " ".join(clean_lines[:10])
        amounts = re.findall(r"([\d,]+\.\d{2})", top_n)
        if amounts:
            # heuristics: often the largest is total due
            try:
                amounts_f = [float(a.replace(',', '')) for a in amounts]
                tot_val = max(amounts_f)
                tot = type(tot)  # keep structure; will use value below
                fields["total_due"] = f"{tot_val:,.2f}"
            except:
                fields["total_due"] = "Not Found"
        else:
            fields["total_due"] = "Not Found"
    else:
        fields["total_due"] = tot.group(1)

    if mind:
        fields["min_due"] = mind.group(1)
    else:
        # try to find a smaller amount near payment block
        if "total_due" in fields and fields["total_due"] != "Not Found":
            try:
                td = float(fields["total_due"].replace(',', ''))
                # look for any amount less than td in top 10 lines
                top_vals = re.findall(r"([\d,]+\.\d{2})", " ".join(clean_lines[:12]))
                top_vals_f = sorted([float(v.replace(',', '')) for v in top_vals])
                candidates = [v for v in top_vals_f if v < td]
                fields["min_due"] = f"{candidates[-1]:,.2f}" if candidates else "Not Found"
            except:
                fields["min_due"] = "Not Found"
        else:
            fields["min_due"] = "Not Found"

    # 8) Credit limit & available credit
    credit_limit = "Not Found"
    available_credit = "Not Found"
    for i, line in enumerate(clean_lines):
        if re.search(r"Credit\s*Limit", line, re.I):
            block = " ".join(clean_lines[i:i+5])
            nums = re.findall(r"([\d,]+(?:\.\d{1,2})?)", block)
            if len(nums) >= 2:
                credit_limit = nums[0]
                available_credit = nums[1]
            elif len(nums) == 1:
                credit_limit = nums[0]
            break
    fields["credit_limit"] = credit_limit
    fields["available_credit"] = available_credit

    return fields


# --- Bank Detection ---
def detect_bank(text):
    # only detect the few banks you treat specially; everything else becomes "OTHER"
    if re.search(r"axis\s*bank", text, re.I):
        return "AXIS"
    elif re.search(r"hdfc", text, re.I):
        return "HDFC"
    elif re.search(r"icici", text, re.I):
        return "ICICI"
    elif re.search(r"idfc", text, re.I):
        return "IDFC"
    else:
        return "OTHER"


# --- MAIN FUNCTION ---
def parse_statement(path):
    text = pdf_to_text(path)
    clean_lines = [line.strip() for line in text.splitlines() if line.strip()]
    bank = detect_bank(text)
    print(f"\n🧾 Detected Bank Flow: {bank}\n")

    if bank == "AXIS":
        fields = parse_axis(text, clean_lines)
        fields["bank"] = "AXIS"
    elif bank == "HDFC":
        fields = parse_hdfc(text, clean_lines)
        fields["bank"] = "HDFC"
    elif bank in ["ICICI", "IDFC"]:
        fields = parse_icici_idfc(text, clean_lines)
        fields["bank"] = bank
    else:
        fields = parse_other_banks(text, clean_lines)
        # D1: prefer canonical bank name if found, else Unknown Bank
        fields["bank"] = fields.get("bank_detected_name", "Unknown Bank")
        # remove auxiliary key to keep output clean
        if "bank_detected_name" in fields:
            del fields["bank_detected_name"]

    return fields


# --- RUN PARSER ---
parsed_fields = parse_statement(pdf_path)
print(json.dumps(parsed_fields, indent=2, ensure_ascii=False))

# --- EXPORT TO EXCEL ---
df = pd.DataFrame([parsed_fields])
excel_path = "statement_fields.xlsx"
df.to_excel(excel_path, index=False)

print(f"\nExcel saved as: {excel_path}")
files.download(excel_path)


Saving icici.pdf to icici (1).pdf
Uploaded: icici (1).pdf

🧾 Detected Bank Flow: ICICI

{
  "card_last4": "8002",
  "payment_due_date": "2018-08-01",
  "total_due": "13,429.57",
  "billing_cycle": "From 14/06/2018 to 13/07/2018",
  "credit_limit": "34,000.00",
  "available_credit": "20,570.43",
  "bank": "ICICI"
}

Excel saved as: statement_fields.xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>